In [ ]:
import mlrun
import mlrun.feature_store as fstore
import queue

In [25]:
%%writefile serving.py

import mlrun
import mlrun.feature_store as fstore
import queue
import datetime
import json
import torch


def postprocess(event):
    print(f"postprocess event {event}")
    return {"prediction":event}


def preprocess(event):
    print(f"init context event {event}")
    tickers_data = {}
    start_time = datetime.datetime.now()-datetime.timedelta(7)
    end_time = datetime.datetime.now()-datetime.timedelta(0)
    train_dataset = fstore.get_offline_features('stocks', entity_timestamp_column='Datetime',  with_indexes=True, start_time=start_time, end_time=end_time)
    price_cols = ['Open','High','Low','Close']
    df = train_dataset.to_dataframe().reset_index(drop=False)
    df.fillna(value=1,inplace=True)
    normalized_df = df.copy()
    tickers = df['ticker'].unique()
    data = []
    labels = []
    normalized_df[price_cols] = (normalized_df[price_cols] - normalized_df[price_cols].mean()) / normalized_df[price_cols].std()
    normalized_df['Volume'] = (normalized_df['Volume'] - normalized_df['Volume'].mean()) / normalized_df['Volume'].std()
    for ticker in tickers:
        ticker_df = normalized_df[normalized_df['ticker'] == ticker].sort_values(by='Datetime').drop(['ticker','Datetime'],axis=1)
        q = queue.Queue()
        for idx,col in ticker_df[-5:].iterrows():
            q.put(col.values.tolist())
        tickers_data[ticker] = q
        
    return torch.tensor([list(value) for key,value in tickers_data.items()]).detach()

Overwriting serving.py


In [2]:
fn = mlrun.code_to_function("testing", kind='nuclio',image='mlrun/mlrun',requirements=['torch'])

ERROR! Session/line number was not unique in database. History logging moved to new session 39


NameError: name 'mlrun' is not defined

In [ ]:
fn.deploy()

In [ ]:
import json
fn.invoke(body=json.dumps({'use_queue':True}),path='http://default-testing-default.default-tenant.app.kjlirbtakxqf.iguazio-cd1.com/')

In [23]:
[i for key,i in {'a':1, 'b':2}.items()]

[1, 2]